In [1]:
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 7.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.9 MB/s eta 0:00:00


# Finetuning GPT on Simpsons Episodes

This is an exercise in using Huggingface to finetune GPT.

In [2]:
from transformers import BertTokenizer, BertForTokenClassification

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForTokenClassification.from_pretrained('bert-base-uncased')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [4]:
input_text = "The quick brown fox jumps over a lazy dog."



In [5]:
inputs = tokenizer(input_text, return_tensors='pt')
outputs = model(**inputs, output_hidden_states=True, output_attentions=True )
last_hidden_states = outputs.hidden_states[-1]

In [6]:
tokens = [tokenizer.convert_ids_to_tokens(x, skip_special_tokens=False) for x in inputs.input_ids.detach().numpy()][0]

print(tokens)

['[CLS]', 'the', 'quick', 'brown', 'fox', 'jumps', 'over', 'a', 'lazy', 'dog', '.', '[SEP]']


In [7]:
tokenizer.convert_ids_to_tokens(7592, skip_special_tokens=False)

'hello'

In [8]:
last_attentions_layer = outputs.attentions[11]

In [9]:
last_attentions_layer[0][10].detach().numpy()

array([[0.03625287, 0.1900843 , 0.01253154, 0.04063654, 0.0272679 ,
        0.28554893, 0.03555759, 0.0763882 , 0.01898166, 0.02135658,
        0.15285458, 0.10253933],
       [0.05941221, 0.07804693, 0.01034812, 0.02422054, 0.01523601,
        0.06809074, 0.00858517, 0.04639668, 0.00669648, 0.00925446,
        0.01123601, 0.6624766 ],
       [0.01340958, 0.01706067, 0.02674684, 0.01866955, 0.01221586,
        0.02141335, 0.0126928 , 0.01274381, 0.01615817, 0.00782855,
        0.00618356, 0.83487725],
       [0.02435662, 0.02133569, 0.0196947 , 0.02119408, 0.01350892,
        0.02990875, 0.0073172 , 0.02201881, 0.01447566, 0.00766142,
        0.0056224 , 0.8129058 ],
       [0.03452975, 0.0198311 , 0.00588981, 0.00656481, 0.00766781,
        0.01971394, 0.00701869, 0.01601711, 0.00435265, 0.00391462,
        0.00726691, 0.86723274],
       [0.05453359, 0.05324076, 0.00927848, 0.02460235, 0.00976239,
        0.04814389, 0.02229482, 0.03776917, 0.02014646, 0.01367278,
        0.00716266,

In [10]:
len(last_attentions_layer[0])

12

In [11]:
import pandas as pd
import numpy as np

square = last_attentions_layer[0][11].detach().numpy()

In [12]:
square[0][1]

0.10934403

In [13]:
df = pd.DataFrame(square)
df.style.background_gradient(cmap ='viridis').set_properties(**{'font-size': '20px'})

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.011482,0.109344,0.039051,0.049422,0.031732,0.081652,0.089833,0.150721,0.063807,0.062535,0.060246,0.250174
1,0.016684,0.022529,0.013882,0.014590,0.020071,0.026084,0.025494,0.021678,0.012902,0.014720,0.004070,0.807296
2,0.016748,0.022678,0.036761,0.026075,0.024403,0.024771,0.040580,0.045715,0.041593,0.019830,0.006309,0.694538
3,0.023997,0.030637,0.025512,0.023776,0.029828,0.029463,0.055109,0.070130,0.047189,0.029030,0.005573,0.629755
4,0.027953,0.042447,0.016062,0.018631,0.049775,0.067566,0.089606,0.092904,0.025417,0.020714,0.009118,0.539808
5,0.015279,0.026837,0.025133,0.012780,0.041319,0.049603,0.064551,0.030697,0.034307,0.044256,0.004082,0.651155
6,0.011938,0.007666,0.010131,0.010125,0.027912,0.033163,0.062022,0.020884,0.041651,0.041675,0.001572,0.731262
7,0.019459,0.020962,0.023791,0.018081,0.047267,0.034987,0.080275,0.052027,0.059809,0.079750,0.002639,0.560953
8,0.018812,0.014660,0.022311,0.013405,0.020821,0.015129,0.059851,0.056523,0.092832,0.092862,0.003614,0.589180
9,0.008607,0.008698,0.012310,0.008279,0.021888,0.025009,0.072626,0.082757,0.081405,0.049992,0.001388,0.627040


In [14]:
def generate_heatmap_data(tokens):
  heatmap_data = []
  for layer in range(len(last_attentions_layer[0])):
    square_2 = last_attentions_layer[0][layer].detach().numpy()
    layer_list = []
    for x in range(len(tokens)):
      for y in range(len(tokens)):
        layer_list.append({'x':tokens[x], 'y':tokens[y], 'color':str(square_2[x][y])})
    heatmap_data.append(layer_list)
  return heatmap_data


In [15]:
data = generate_heatmap_data(tokens)

In [16]:
data

[[{'x': '[CLS]', 'y': '[CLS]', 'color': '0.25109136'},
  {'x': '[CLS]', 'y': 'the', 'color': '0.061356496'},
  {'x': '[CLS]', 'y': 'quick', 'color': '0.013422967'},
  {'x': '[CLS]', 'y': 'brown', 'color': '0.01561833'},
  {'x': '[CLS]', 'y': 'fox', 'color': '0.032868307'},
  {'x': '[CLS]', 'y': 'jumps', 'color': '0.18642499'},
  {'x': '[CLS]', 'y': 'over', 'color': '0.059978172'},
  {'x': '[CLS]', 'y': 'a', 'color': '0.07427802'},
  {'x': '[CLS]', 'y': 'lazy', 'color': '0.03753171'},
  {'x': '[CLS]', 'y': 'dog', 'color': '0.021716587'},
  {'x': '[CLS]', 'y': '.', 'color': '0.010082211'},
  {'x': '[CLS]', 'y': '[SEP]', 'color': '0.23563077'},
  {'x': 'the', 'y': '[CLS]', 'color': '0.029874215'},
  {'x': 'the', 'y': 'the', 'color': '0.015493642'},
  {'x': 'the', 'y': 'quick', 'color': '0.0146139935'},
  {'x': 'the', 'y': 'brown', 'color': '0.0077904994'},
  {'x': 'the', 'y': 'fox', 'color': '0.008348609'},
  {'x': 'the', 'y': 'jumps', 'color': '0.053882014'},
  {'x': 'the', 'y': 'over', 

In [17]:
import json

dumps = json.dumps(data, indent=4)  

with open("bert_heatmap.json", "w") as outfile:
    outfile.write(dumps)